In [ ]:
import os
seed = 0
os.environ['PYTHONHASSEED'] = str(seed)

In [ ]:
# Disable GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import pickle
from decimal import Decimal

from tensorflow.random import set_seed

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [ ]:
# Set seed
random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [ ]:
filename = '../data/turbofan_dataset/N-CMAPSS_DS02-006.h5'
output_path = 'DS02/experiment_set_7'

In [ ]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
def load_dataset(filename):
    """ Reads a dataset from a given .h5 file and compose (in memory) the train and test data. 
    Args:
        filename(str): path to the .h5 file
    Returns:
        train_set(pd.DataFrame), test_set(pd.DataFrame)
    """
    with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        columns = []
        columns += list(np.array(A_var, dtype='U20'))
        columns += list(np.array(T_var, dtype='U20'))
        columns += list(np.array(X_s_var, dtype='U20'))
        columns += list(np.array(X_v_var, dtype='U20'))
        columns += list(np.array(W_var, dtype='U20'))
        columns += ['RUL']
        
    train_set = np.concatenate((A_dev, T_dev, X_s_dev, X_v_dev, W_dev, Y_dev), axis=1)
    test_set = np.concatenate((A_test, T_test, X_s_test, X_v_test, W_test, Y_test), axis=1)
    
    return pd.DataFrame(data=train_set, columns=columns), pd.DataFrame(data=test_set, columns=columns)

In [ ]:
start_time = time.process_time()  
train_set, test_set = load_dataset(filename)
print('')
print("Operation time (sec): " , (time.process_time() - start_time))
print('')
print ("Train set shape: " + str(train_set.shape))
print ("Test set shape: " + str(test_set.shape))

In [ ]:
def unit_cycle_info(df, compute_cycle_len=False):
    unit_ids = np.unique(df['unit'])
    print('Engine units in df: ', unit_ids)
    for i in unit_ids:
        num_cycles = len(np.unique(df.loc[df['unit'] == i, 'cycle']))
        print('Unit: ', i, ' - Number of flight cycles: ', num_cycles)
        
    if compute_cycle_len:
        cycle_ids = np.unique(df['cycle'])
        print('Total number of cycles: ', len(cycle_ids))
        min_len = np.inf
        max_len = 0
        for i in cycle_ids:
            cycle_len = len(df.loc[df['cycle'] == i, 'cycle'])
            if cycle_len < min_len:
                min_len = cycle_len
            elif cycle_len > max_len:
                max_len = cycle_len
        print('Min cycle length: ', min_len)
        print('Max cycle length: ', max_len)
    
    return unit_ids

In [ ]:
# Filter constant and quasi-constant features
def get_quasi_constant_features(dataset, variance_th=0.01, debug=True):
    constant_filter = VarianceThreshold(threshold=variance_th)
    constant_filter.fit(dataset)
    constant_features = [col for col in dataset.columns 
                         if col not in dataset.columns[constant_filter.get_support()]]
    
    if debug:
        print("Number of non-constant features: ", len(dataset.columns[constant_filter.get_support()]))
        
        print("Number of quasi-constant features: ", len(constant_features))
        print("Quasi-constant features: ")
        for col in constant_features:
            print(col)
    return constant_features

def get_non_correlated_features(dataset, corr_th=0.9, debug=True):
    corr_mat = dataset.corr()
    corr_mat = np.abs(corr_mat)
    
    N = corr_mat.shape[0]
    columns = np.full((N,), True, dtype=bool)
    for i in range(N):
        for j in range(i+1, N):
            if corr_mat.iloc[i, j] >= corr_th:
                if columns[j]:
                    columns[j] = False
    if debug:        
        correlated_features = dataset.columns[~columns]
        print("Number of correlated features: ", len(correlated_features))
        print("Correlated features: ")
        for col in correlated_features:
            print(col)
    
    selected_columns = dataset.columns[columns]
    return selected_columns

In [ ]:
train_unit_ids = unit_cycle_info(train_set)
test_unit_ids = unit_cycle_info(test_set)

In [ ]:
y_train = train_set['RUL']
x_train = train_set.drop(['RUL'], axis=1)

In [ ]:
constant_features = get_quasi_constant_features(x_train)
x_train.drop(labels=constant_features, axis=1, inplace=True)
print("Train shape: ", x_train.shape)

In [ ]:
# Remove highly correlated features
selected_columns = get_non_correlated_features(x_train)
x_train = x_train[selected_columns]
print("Train shape: ", x_train.shape)

In [15]:
# Drop auxiliary data columns
auxiliary_columns = ['cycle', 'hs', 'Fc']
x_train.drop(labels=[x for x in auxiliary_columns if x in x_train.columns], axis=1, inplace=True)

In [16]:
y_test = test_set['RUL']
x_test = test_set.drop(['RUL'], axis=1)
x_test = x_test[x_train.columns]

In [17]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')

In [18]:
W = 10
col_names = [col for col in x_train.columns if col != 'unit']
print('Columns: ', col_names)
print('Train unit ids: ', train_unit_ids)
print('Test unit ids: ', test_unit_ids)

Columns:  ['T24', 'T30', 'P15', 'SmFan', 'SmLPC']
Train unit ids:  [ 2.  5. 10. 16. 18. 20.]
Test unit ids:  [11. 14. 15.]


In [19]:
def time_window_processing(X, y, column_names, unit_ids, window_size):
    df = pd.concat([X, y], axis=1)
    lag_columns = []
    
    for col_name in column_names:
        for i in range(1, window_size):
            partial_columns = []
            for j in range(len(unit_ids)):
                unit_df = df.loc[df['unit'] == unit_ids[j], :]
                col = unit_df[col_name].shift(i)
                col.name = '{}(t-{})'.format(col_name, i)
                partial_columns.append(col)
            column = pd.concat(partial_columns)
            lag_columns.append(column)
    
    df = pd.concat([df] + lag_columns, axis=1)
    df.dropna(inplace=True)
    df.drop(labels=['unit'], axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    feature_columns = [col for col in df.columns.values if col != 'RUL']    
    return df[feature_columns], df['RUL']

In [22]:
x_train_tw, y_train_tw = time_window_processing(x_train, y_train, col_names, train_unit_ids, W)
x_test_tw, y_test_tw = time_window_processing(x_test, y_test, col_names, test_unit_ids, W)

In [ ]:
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_tw, y_train_tw, 
                                                                          test_size=0.3, 
                                                                          random_state=seed)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_split)
x_val_scaled = scaler.transform(x_val_split)
x_test_scaled = scaler.transform(x_test_tw)

In [ ]:
x_train_final, y_train_final = x_train_scaled, y_train_split
x_val_final, y_val_final = x_val_scaled, y_val_split
x_test_final, y_test_final = x_test_scaled, y_test_tw

In [24]:
def cmapss_score_function(actual, predictions, normalize=True):
    # diff < 0 -> over-estimation
    # diff > 0 -> under-estimation
    diff = actual - predictions
    alpha = np.full_like(diff, 1/13)
    negative_diff_mask = diff < 0
    alpha[negative_diff_mask] = 1/10
    score = np.sum(np.exp(alpha * np.abs(diff)))
    
    if normalize:
        N = len(predictions)
        score /= N
    return score

def compute_evaluation_metrics(actual, predictions, label='Test'):
    mse = mean_squared_error(actual, predictions)
    rmse = np.sqrt(mse)
    cmapss_score = cmapss_score_function(actual, predictions)
    print('{} set:\nMSE: {:.2f}\nRMSE: {:.2f}\nCMAPSS score: {:.2f}\n'.format(label, mse, rmse, 
                                                                     Decimal(cmapss_score)))
    return mse, rmse, cmapss_score
    
def plot_loss_curves(history, output_path=None, y_lim=[0, 150]):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.ylim(y_lim)
    plt.legend(['train', 'validation'], loc='upper left')
    
    if output_path is not None:
        plt.savefig(os.path.join(output_path, 'loss_curves.png'), format='png', dpi=300) 
    plt.show()
    
def plot_rul(expected, predicted):
    plt.figure()
    plt.plot(range(len(expected)), expected, label='Expected')
    plt.plot(range(len(predicted)), predicted, label='Predicted')
    plt.legend()
    
    
def create_mlp_model(input_dim, hidden_layer_sizes, activation='relu', output_weights_file=None):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], 
                    input_dim=input_dim, 
                    kernel_initializer='random_normal', 
                    activation=activation))

    for layer_size in hidden_layer_sizes[1:]:
        model.add(Dense(layer_size, 
                        kernel_initializer='random_normal', 
                        activation=activation))
    
    model.add(Dense(1, kernel_initializer='random_normal'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    if output_weights_file is not None:
        model.save_weights(output_weights_file)
    return model

def train_model_existing_weights(model, weights_file, x_train, y_train, x_val, y_val, epochs=200, batch_size=512, callbacks=[]):
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.load_weights(weights_file)
    return model.fit(x_train, y_train,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     batch_size=batch_size,
                     verbose=1,
                     callbacks=callbacks)

def save_history(history, output_file=os.path.join(output_path, "history.pkl")):
    with open(output_file, 'wb') as file:
        pickle.dump(history.history, file)
    print("Saved training history to file: {}".format(output_file))

def load_history(file):
    return pickle.load(open(file, "rb"))

def model_evaluation(model, x_test, y_test, x_train=None, y_train=None, plot_range=[0, 10**3]):
    if x_train is not None and y_train is not None:
        predictions_train = model.predict(x_train).flatten()
        compute_evaluation_metrics(predictions_train, y_train, 'Train')
        
        expected = y_train[plot_range[0]:plot_range[1]]
        predicted = predictions_train[plot_range[0]:plot_range[1]]
        plot_rul(expected, predicted)
        
    predictions_test = model.predict(x_test).flatten()
    compute_evaluation_metrics(predictions_test, y_test)
    
    expected = y_test[plot_range[0]:plot_range[1]]
    predicted = predictions_test[plot_range[0]:plot_range[1]]
    plot_rul(expected, predicted)

In [ ]:
# Retrain best model in experiment set 4
weights_file = r"D:\Licenta\notebooks\DS02\experiment_set_7\mlp_weights.h5"
batch_size = 512
epochs = 200

layer_sizes = [256, 256, 512, 64]
input_dim = x_train_final.shape[1]

model = create_mlp_model(input_dim, layer_sizes, 
                         output_weights_file=weights_file, 
                         activation='tanh')
model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint(os.path.join(output_path, 'mlp_model_trained.h5'), 
                     monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = train_model_existing_weights(model, weights_file, x_train_final, y_train_final, 
                                       x_val_final, y_val_final, 
                                       batch_size=batch_size, 
                                       epochs=epochs, 
                                       callbacks=[es, mc])

save_history(history)

In [ ]:
history_path = r'D:\Licenta\notebooks\DS02\experiment_set_7\history.pkl'
model_path = r'D:\Licenta\notebooks\DS02\experiment_set_7\mlp_model_trained.h5'

history = load_history(history_path)
plot_loss_curves(history, output_path, y_lim=[0, 120])

In [ ]:
loaded_model = load_model(model_path)
model_evaluation(loaded_model, x_test=x_test_final, y_test=y_test_final, 
                 x_train=x_train_final, y_train=y_train_final)

In [26]:
######################
# Test multiple splits
######################
weights_file = r"D:\Licenta\notebooks\DS02\experiment_set_7\window_{}\mlp_weights.h5".format(W)
batch_size = 512
epochs = 200

layer_sizes = [256, 256, 512, 64]
input_dim = x_train_tw.shape[1]

results = []

for random_seed in range(1, 5):
    model_path = os.path.join(output_path, "window_{}".format(W), 
                              "split_seed_{}".format(random_seed))
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    
    x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_tw, y_train_tw, 
                                                                              test_size=0.3, 
                                                                              random_state=random_seed)
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train_split)
    x_val_scaled = scaler.transform(x_val_split)

    model = create_mlp_model(input_dim, layer_sizes, activation='tanh')
    model.summary()
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint(os.path.join(model_path, 'mlp_model_trained.h5'),
                         monitor='val_loss', mode='min', verbose=1, save_best_only=True)

    history = train_model_existing_weights(model, weights_file, 
                                           x_train_scaled, y_train_split, 
                                           x_val_scaled, y_val_split, 
                                           batch_size=batch_size, 
                                           epochs=epochs, 
                                           callbacks=[es, mc])
    
    history_path = os.path.join(model_path, "history.pkl")
    save_history(history, history_path)
    results.append((model, history, scaler))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               13056     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 243,329
Trainable params: 243,329
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
7191/7197 [============================>.] - ETA: 0s - loss: 173.9858
Epoch 00001: val_loss improved f

7187/7197 [============================>.] - ETA: 0s - loss: 25.3885
Epoch 00029: val_loss did not improve from 22.54233
7197/7197 [==============================] - 34s 5ms/step - loss: 25.3870 - val_loss: 50.5568
Epoch 30/200
7195/7197 [============================>.] - ETA: 0s - loss: 25.1237
Epoch 00030: val_loss did not improve from 22.54233
7197/7197 [==============================] - 34s 5ms/step - loss: 25.1256 - val_loss: 33.7644
Epoch 31/200
7189/7197 [============================>.] - ETA: 0s - loss: 24.9809
Epoch 00031: val_loss did not improve from 22.54233
7197/7197 [==============================] - 34s 5ms/step - loss: 24.9791 - val_loss: 24.6285
Epoch 32/200
7196/7197 [============================>.] - ETA: 0s - loss: 24.6521
Epoch 00032: val_loss did not improve from 22.54233
7197/7197 [==============================] - 34s 5ms/step - loss: 24.6520 - val_loss: 27.3339
Epoch 33/200
7187/7197 [============================>.] - ETA: 0s - loss: 24.3032
Epoch 00033: val_lo

Epoch 23/200
7196/7197 [============================>.] - ETA: 0s - loss: 27.2378
Epoch 00023: val_loss did not improve from 26.40287
7197/7197 [==============================] - 34s 5ms/step - loss: 27.2379 - val_loss: 38.1242
Epoch 24/200
7195/7197 [============================>.] - ETA: 0s - loss: 26.9348
Epoch 00024: val_loss did not improve from 26.40287
7197/7197 [==============================] - 33s 5ms/step - loss: 26.9358 - val_loss: 35.1895
Epoch 25/200
7193/7197 [============================>.] - ETA: 0s - loss: 26.5440
Epoch 00025: val_loss did not improve from 26.40287
7197/7197 [==============================] - 34s 5ms/step - loss: 26.5424 - val_loss: 26.9771
Epoch 26/200
7197/7197 [==============================] - ETA: 0s - loss: 26.4466
Epoch 00026: val_loss did not improve from 26.40287
7197/7197 [==============================] - 35s 5ms/step - loss: 26.4466 - val_loss: 30.6188
Epoch 27/200
7189/7197 [============================>.] - ETA: 0s - loss: 26.0374
Epoch 

Epoch 57/200
7187/7197 [============================>.] - ETA: 0s - loss: 20.9087
Epoch 00057: val_loss did not improve from 20.37637
7197/7197 [==============================] - 36s 5ms/step - loss: 20.9180 - val_loss: 28.4781
Epoch 00057: early stopping
Saved training history to file: DS02/experiment_set_7\window_10\split_seed_2\history.pkl
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               13056     
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                32832     
_______________________________________

Epoch 28/200
7190/7197 [============================>.] - ETA: 0s - loss: 24.7775
Epoch 00028: val_loss did not improve from 24.83752
7197/7197 [==============================] - 36s 5ms/step - loss: 24.7798 - val_loss: 26.5689
Epoch 29/200
7186/7197 [============================>.] - ETA: 0s - loss: 24.4736
Epoch 00029: val_loss improved from 24.83752 to 21.71376, saving model to DS02/experiment_set_7\window_10\split_seed_3\mlp_model_trained.h5
7197/7197 [==============================] - 37s 5ms/step - loss: 24.4678 - val_loss: 21.7138
Epoch 30/200
7196/7197 [============================>.] - ETA: 0s - loss: 24.3122
Epoch 00030: val_loss did not improve from 21.71376
7197/7197 [==============================] - 36s 5ms/step - loss: 24.3122 - val_loss: 43.8968
Epoch 31/200
7195/7197 [============================>.] - ETA: 0s - loss: 24.2436
Epoch 00031: val_loss did not improve from 21.71376
7197/7197 [==============================] - 35s 5ms/step - loss: 24.2428 - val_loss: 27.7922


7197/7197 [==============================] - 36s 5ms/step - loss: 30.7116 - val_loss: 28.6187
Epoch 17/200
7190/7197 [============================>.] - ETA: 0s - loss: 29.8284
Epoch 00017: val_loss did not improve from 28.61875
7197/7197 [==============================] - 37s 5ms/step - loss: 29.8322 - val_loss: 43.9378
Epoch 18/200
7190/7197 [============================>.] - ETA: 0s - loss: 29.5876
Epoch 00018: val_loss improved from 28.61875 to 26.54677, saving model to DS02/experiment_set_7\window_10\split_seed_4\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 29.5865 - val_loss: 26.5468
Epoch 19/200
7194/7197 [============================>.] - ETA: 0s - loss: 29.0122
Epoch 00019: val_loss did not improve from 26.54677
7197/7197 [==============================] - 36s 5ms/step - loss: 29.0124 - val_loss: 33.6148
Epoch 20/200
7190/7197 [============================>.] - ETA: 0s - loss: 28.5390
Epoch 00020: val_loss improved from 26.54677 to 26.2

Epoch 50/200
7194/7197 [============================>.] - ETA: 0s - loss: 21.3122
Epoch 00050: val_loss did not improve from 19.09345
7197/7197 [==============================] - 37s 5ms/step - loss: 21.3108 - val_loss: 19.5811
Epoch 51/200
7194/7197 [============================>.] - ETA: 0s - loss: 21.3922
Epoch 00051: val_loss did not improve from 19.09345
7197/7197 [==============================] - 37s 5ms/step - loss: 21.3920 - val_loss: 26.5148
Epoch 52/200
7192/7197 [============================>.] - ETA: 0s - loss: 21.2211
Epoch 00052: val_loss did not improve from 19.09345
7197/7197 [==============================] - 37s 5ms/step - loss: 21.2223 - val_loss: 21.6264
Epoch 53/200
7186/7197 [============================>.] - ETA: 0s - loss: 20.9212
Epoch 00053: val_loss did not improve from 19.09345
7197/7197 [==============================] - 37s 5ms/step - loss: 20.9284 - val_loss: 36.9485
Epoch 54/200
7189/7197 [============================>.] - ETA: 0s - loss: 20.9149
Epoch 

In [28]:
mse_vals = []
rmse_vals = []
cmapss_score_vals = []

for i in range(1, 5):
    model_path = os.path.join(output_path, "window_{}".format(W), "split_seed_{}".format(i))
    model = load_model(os.path.join(model_path, "mlp_model_trained.h5"))
    scaler = results[i - 1][2]
    x_test_scaled = scaler.transform(x_test_tw)
    predictions_test = model.predict(x_test_scaled).flatten()
    mse, rmse, cmapss_score = compute_evaluation_metrics(predictions_test, y_test_tw)
    mse_vals.append(mse)
    rmse_vals.append(rmse)
    cmapss_score_vals.append(cmapss_score)
    
print(mse_vals)
print(rmse_vals)
print(cmapss_score_vals)

Test set:
MSE: 62.69
RMSE: 7.92
CMAPSS score: 1.80

Test set:
MSE: 64.41
RMSE: 8.03
CMAPSS score: 1.81

Test set:
MSE: 67.55
RMSE: 8.22
CMAPSS score: 1.84

Test set:
MSE: 66.79
RMSE: 8.17
CMAPSS score: 1.85

[62.69164, 64.40734, 67.55202, 66.78518]
[7.917805, 8.025418, 8.219004, 8.17222]
[1.7950371136684864, 1.8138475938729346, 1.8381098669874198, 1.8468223664689611]


In [29]:
def print_summary_statistics(arr):
    print("Mean: {:.2f}".format(np.mean(arr)))
    print("Stddev: {:.2f}".format(np.std(arr)))

In [30]:
# For random seed 0
mse_vals += [64.41]
rmse_vals += [8.03]
cmapss_score_vals += [1.83]

In [31]:
print("MSE: ")
print_summary_statistics(mse_vals)

print("\nRMSE: ")
print_summary_statistics(rmse_vals)

print("\nCMAPSS score: ")
print_summary_statistics(cmapss_score_vals)

MSE: 
Mean: 65.17
Stddev: 1.77

RMSE: 
Mean: 8.07
Stddev: 0.11

CMAPSS score: 
Mean: 1.82
Stddev: 0.02
